In [1]:
import pickle
import numpy as np
from keras.utils import np_utils

In [2]:
with open("/content/notes",'rb') as f:
  notes = pickle.load(f)

In [3]:
n_vocab = len(set(notes))

In [4]:
print("Total notes: ",len(notes))
print("Unique notes: ",n_vocab)

Total notes:  11358
Unique notes:  197


## Prepare Sequential data for LSTM

In [5]:
#how many elements LSTM input should consider
sequence_len = 100

In [6]:
pitchnames = sorted(set(notes))

In [7]:
#mapping between ele to int value
ele_to_int = dict((ele,num) for num,ele in enumerate(pitchnames))

In [8]:
network_input = []
network_output = []

In [9]:
for i in range(len(notes) - sequence_len):
    seq_in = notes[i:i+sequence_len]
    seq_out = notes[i+sequence_len]
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [10]:
# No. of examples

n_patterns = len(network_input)
print(n_patterns)

11258


In [11]:
network_input = np.reshape(network_input, (n_patterns, sequence_len,1))

In [12]:
print(network_input.shape)

(11258, 100, 1)


In [13]:
normalized_network_input = network_input/float(n_vocab)

In [14]:
normalized_network_input

array([[[0.77664975],
        [0.97461929],
        [0.97969543],
        ...,
        [0.56852792],
        [0.13705584],
        [0.17766497]],

       [[0.97461929],
        [0.97969543],
        [0.67005076],
        ...,
        [0.13705584],
        [0.17766497],
        [0.58375635]],

       [[0.97969543],
        [0.67005076],
        [0.74619289],
        ...,
        [0.17766497],
        [0.58375635],
        [0.56852792]],

       ...,

       [[0.58375635],
        [0.66497462],
        [0.84263959],
        ...,
        [0.85786802],
        [0.60406091],
        [0.75126904]],

       [[0.66497462],
        [0.84263959],
        [0.74111675],
        ...,
        [0.60406091],
        [0.75126904],
        [0.86294416]],

       [[0.84263959],
        [0.74111675],
        [0.58883249],
        ...,
        [0.75126904],
        [0.86294416],
        [0.60913706]]])

In [15]:
# Network output are the classes, encode into one hot vector

In [16]:
network_output = np_utils.to_categorical(network_output)

In [17]:
network_output.shape

(11258, 197)

In [18]:
print("Input data shape: ",normalized_network_input.shape)
print("Output data shape: ",network_output.shape)

Input data shape:  (11258, 100, 1)
Output data shape:  (11258, 197)


## Create Model

In [19]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [20]:
model = Sequential()
model.add(LSTM(units = 512,
               input_shape = (normalized_network_input.shape[1],normalized_network_input.shape[2]),
               return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [22]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')

In [23]:
checkpoint = ModelCheckpoint('model.hdf5',monitor = 'loss',verbose=0,save_best_only=True,mode = 'min')
model_his = model.fit(normalized_network_input,network_output,epochs = 100,batch_size = 64, callbacks = [checkpoint])

Epoch 1/100
176/176 [==============================] - 23s 74ms/step - loss: 4.7026
Epoch 2/100
176/176 [==============================] - 13s 75ms/step - loss: 4.4965
Epoch 3/100
176/176 [==============================] - 13s 76ms/step - loss: 4.4936
Epoch 4/100
176/176 [==============================] - 14s 77ms/step - loss: 4.4409
Epoch 5/100
176/176 [==============================] - 14s 78ms/step - loss: 4.4069
Epoch 6/100
176/176 [==============================] - 14s 80ms/step - loss: 4.3868
Epoch 7/100
176/176 [==============================] - 14s 81ms/step - loss: 4.3652
Epoch 8/100
176/176 [==============================] - 14s 82ms/step - loss: 4.3342
Epoch 9/100
176/176 [==============================] - 14s 81ms/step - loss: 4.3301
Epoch 10/100
176/176 [==============================] - 14s 81ms/step - loss: 4.3101
Epoch 11/100
176/176 [==============================] - 14s 80ms/step - loss: 4.2963
Epoch 12/100
176/176 [==============================] - 14s 81ms/step - lo

In [24]:
from google.colab import files

In [25]:
files.download('/content/model.hdf5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>